### Stock Price Forecasting
- Goal: Predict next-day stock price from past sequence data.
- Dataset: Yahoo Finance (https://pypi.org/project/yfinance/)
- Model: LSTM for regression
- Task Type: Regression
- Extension: Add multi-step forecasting; compare CNN vs. LSTM.

In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt 
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset, TensorDataset
from torch.utils.tensorboard import SummaryWriter
import yfinance as yf
from LSTM import SimpleLSTM

In [33]:
symbols = ['MSTF', 'GOOGL', 'AMZN', 'TSLA']
for symbol in symbols:
    stock = yf.Ticker(symbol)
    info = stock.info
    print(f"Symbol: {symbol}")
    print(f"Name: {info.get('shortName', 'N/A')}")
    print(f"Market: {info.get('market', 'N/A')}")
    print(f"Sector: {info.get('sector', 'N/A')}")
    print("-" * 30)

Symbol: MSTF
Name: MONARCH STAFFING INC
Market: us_market
Sector: N/A
------------------------------
Symbol: GOOGL
Name: Alphabet Inc.
Market: us_market
Sector: Communication Services
------------------------------
Symbol: AMZN
Name: Amazon.com, Inc.
Market: us_market
Sector: Consumer Cyclical
------------------------------
Symbol: TSLA
Name: Tesla, Inc.
Market: us_market
Sector: Consumer Cyclical
------------------------------


In [34]:
# Tai toan bo du lieu tu 5 nam tro lai
dat = yf.download('AMZN', period='5y')
# Luu vao file
dat.to_csv('AMZN_5y.csv')



C:\Users\Admin\AppData\Local\Temp\ipykernel_18596\3028998652.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  dat = yf.download('AMZN', period='5y')
[*********************100%***********************]  1 of 1 completed


In [35]:
df = pd.read_csv('AMZN_5y.csv')
print(df.columns)


Index(['Price', 'Close', 'High', 'Low', 'Open', 'Volume'], dtype='object')


In [36]:
df.head()

,Price,Close,High,Low,Open,Volume
0,Ticker,AMZN,AMZN,AMZN,AMZN,AMZN
1,Date,NaN,NaN,NaN,NaN,NaN
2,2020-11-06,165.56849670410156,166.10000610351562,161.60000610351562,165.23199462890625,92946000
3,2020-11-09,157.18699645996094,164.4499969482422,155.60549926757812,161.55149841308594,143808000
4,2020-11-10,151.75100708007812,155.6999969482422,150.9739990234375,154.75100708007812,131820000


In [37]:
df.isnull().sum()

Price     0
Close     1
High      1
Low       1
Open      1
Volume    1
dtype: int64